In [2]:
import os
import pandas as pd

In [3]:
output_files = !ls ./amplicon_outputs/*.csv

primer_files = !ls ./primer_inputs/*.csv

In [4]:
primer_df = pd.DataFrame([])

for primer_file in primer_files:
  exp_name = os.path.splitext(os.path.basename(primer_file))[0].rstrip("_primer_stats")
  df = pd.read_csv(primer_file,
                   usecols=['Primer_Name', 'Seq', 'Amplicon',
                            'Primer_Length', 'Tm', 'Delta Tm',
                            'GC_Content', 'Length_Longest_Homopolymer', 'Percent_Homopolymer'])
  df['exp'] = exp_name
  primer_df = primer_df.append(df)

In [5]:
primer_df['flank'] = primer_df['Primer_Name'].apply(lambda x: x.split("_")[-1])

In [6]:
forward = primer_df[primer_df.flank == "F"]
reverse = primer_df[primer_df.flank == "R"]

In [7]:
primers = forward.merge(reverse, left_on=["Amplicon", 'exp'], right_on=["Amplicon", "exp"], suffixes=('_L', '_R'))


In [8]:
output_df = pd.DataFrame([])

for output_file in output_files:
    df = pd.read_csv(output_file)
    output_df = output_df.append(df)
    
output_df.head()
output_df = output_df.rename(columns = {'amplicon_id' : 'Amplicon'})


In [9]:
merged_df = primers.merge(output_df, left_on = 'Amplicon', right_on = 'Amplicon')
merged_df = merged_df.sort_values('Amplicon')

merged_df = merged_df[merged_df['strain'].isna()]

#merged_df.to_csv(index = False)